In [1]:
import os
import pandas as pd
from elasticsearch import Elasticsearch
from dotenv import load_dotenv

load_dotenv()

True

In [2]:
ip_address = os.getenv('IP_ADDRESS')
username = os.getenv('USERNAME')
password = os.getenv('PASSWORD')

ip_address, username, password

('http://localhost:9200/', 'admin', 'admin123')

In [3]:
documents_path = 'E:/Financial_Chatbot/data/finance_department.xlsx'
index_name_nosyn = 'finance_department'
index_name_syn = 'finance_department_syn'

In [4]:
es = Elasticsearch(
    [ip_address],
    basic_auth=(username, password),
    verify_certs=False,
    ssl_show_warn=False
)

if es.ping():
    print("=" * 100)
    print("ElasticSearch Connection Status:")
    print(f"\t Elasticsearch info: {es.info()}")
    print("=" * 100)

ElasticSearch Connection Status:
	 Elasticsearch info: {'name': 'A-KAVIAN-10B', 'cluster_name': 'elasticsearch', 'cluster_uuid': 'XiiNpAHnSeqqZMKTVOITkQ', 'version': {'number': '8.15.0', 'build_flavor': 'default', 'build_type': 'zip', 'build_hash': '1a77947f34deddb41af25e6f0ddb8e830159c179', 'build_date': '2024-08-05T10:05:34.233336849Z', 'build_snapshot': False, 'lucene_version': '9.11.1', 'minimum_wire_compatibility_version': '7.17.0', 'minimum_index_compatibility_version': '7.0.0'}, 'tagline': 'You Know, for Search'}


In [5]:
data = pd.read_excel(documents_path, header=None)
data = data[0].tolist()
data

['دامنه کاربرد این دستورالعمل شامل کلیه کارکنان قراردادی تمام وقت شرکت خدمات انفورماتیک می\xadباشد.',
 'تسهیلات اضطراری: این تسهیلات برای رفع نیازهای اضطراری و فوری کارکنان با سقف 1,000,000,000 ریال، بازپرداخت 36 ماهه و بدون سود و کارمزد ارائه می\u200cشود. تضمین مورد نیاز آن، سفته معادل مبلغ تسهیلات به همراه 2 نفر ضامن است.',
 'تسهیلات ضروری: این تسهیلات برای رفع نیازهای ضروری کارکنان، از جمله تعمیر ساختمان و خرید خودرو، با سقف 4,000,000,000 ریال و بازپرداخت 72 ماهه و بدون سود و کارمزد ارائه می\u200cشود. تضمین آن، سفته معادل مبلغ تسهیلات با 4 نفر ضامن است.',
 'تسهیلات ازدواج: این تسهیلات به منظور تسهیل سنت حسنه ازدواج با سقف 3,000,000,000 ریال، بازپرداخت 72 ماهه و بدون سود و کارمزد ارائه می\u200cشود. برای این تسهیلات، سفته معادل مبلغ تسهیلات و 2 نفر ضامن لازم است.',
 'تسهیلات مسکن (خرید) یا خرید خانه: این تسهیلات به منظور خرید منزل مسکونی با سقف 7,000,000,000 ریال و بازپرداخت 120 ماهه و بدون سود و کارمزد ارائه می\u200cشود. تضمین آن شامل سفته معادل 1.5 برابر مبلغ تسهیلات و 4 نفر ضامن اس

In [8]:
index_config = {
    "mappings": {
        "properties": {"content": {"type": "text"},
        }
    }
}

try:
    es.indices.create(index=index_name_nosyn, body=index_config)
    print(f"--------------------- Index {index_name_nosyn} created successfully ---------------------")
except Exception as e:
    print("--------------------- Can't create index >>> ", e)

--------------------- Can't create index >>>  BadRequestError(400, 'resource_already_exists_exception', 'index [finance_department/U4hmFm_6Qf-AFX0VcCA7vQ] already exists')


In [ ]:
for i, doc in enumerate(data):
    doc_body = {
        "content": doc,
    }
    unique_id = str(i)
    es.index(index=index_name_nosyn, id=unique_id, body=doc_body, refresh='wait_for')
    print(i)
print("--------------------- index successfully created ---------------------")

In [9]:
def query_documents(query):
    k=60
    text_search_query = {
        "size": 5,
        "query": {
            "match": {
                "content": query
            }
        }
    }
    return es.search(index=index_name_nosyn, body=text_search_query)['hits']['hits']

In [10]:
query = 'وام خونه'
query_documents(query)

[{'_index': 'finance_department',
  '_id': '54',
  '_score': 2.8450253,
  '_source': {'content': 'چهار نفر از کارکنان شرکت که مبلغ حقوق آن\u200cها کافی برای پوشش اقساط وام مسکن است، باید ضمانت پرداخت اقساط تسهیلات را بپذیرند.'}},
 {'_index': 'finance_department',
  '_id': '24',
  '_score': 2.7138984,
  '_source': {'content': 'کارکنانی که تسهیلات ضروری دریافت کرده\u200cاند، می\u200cتوانند پس از پرداخت دو سوم از کل اقساط تعیین\u200cشده، درخواست تجدید تسهیلات کنند یا برای وام جدید درخواست بدهند.'}},
 {'_index': 'finance_department',
  '_id': '63',
  '_score': 2.3524802,
  '_source': {'content': 'استفاده از تضامین وام\u200cهای قبلی فرد اعم از شرکت و یا صندوق پس\u200cانداز کارکنان، در صورت تأیید مدیریت مالی و پوشش مانده سایر تسهیلات فرد، قابل استفاده برای وام\u200cهای جدید، صرف\u200cنظر از عنوان وام خواهد بود.'}},
 {'_index': 'finance_department',
  '_id': '20',
  '_score': 1.897603,
  '_source': {'content': 'سقف تسهیلات اضطراری معادل 1,000,000,000 ریال است و میزان یا مقدار وام اظطراری بر ا

In [11]:
def drop_index(index_name):
    try:
        es.indices.delete(index=index_name)
        print(f"Index {index_name} deleted successfully.")
    except Exception as e:
        print(f"Can't Drop {index_name}: ", e)

In [13]:
synonyms_list = pd.read_csv('E:/Financial_Chatbot/Synonyms/synonyms.csv')
synonyms_list = synonyms_list['Synonyms'].to_list()
len(synonyms_list)

19891

In [14]:
synonyms = [
    "کارکنان قراردادی, کارمندان قراردادی, نیروی قراردادی, کارکنان شرکت",
    "تسهیلات اضطراری, وام اضطراری, کمک اضطراری, تسهیلات فوری",
    "تسهیلات ضروری, وام ضروری, تسهیلات ضروری, نیازهای ضروری",
    "تسهیلات ازدواج, وام ازدواج, کمک‌هزینه ازدواج, تسهیلات ازدواجی",
    "تسهیلات مسکن خرید, وام مسکن خرید, کمک خرید خانه, تسهیلات مسکن",
    "تسهیلات مسکن ساخت, وام ساخت, کمک‌هزینه ساخت, تسهیلات ساخت خانه",
    "تسهیلات مسکن رهن, وام رهن, کمک رهن خانه, تسهیلات ودیعه مسکن",
    "خانه, منزل, مسکن, ملک, واحد مسکونی",
    "خرید خانه, خرید ملک, خرید منزل, خرید مسکن",
    "ساخت خانه, ساخت ملک, ساخت منزل, پروژه مسکونی",
    "رهن خانه, ودیعه مسکن, اجاره خانه, رهن منزل",
    "سابقه خدمت, سابقه کاری, مدت کار, تجربه کاری",
    "امتیاز عملکرد, نمره عملکرد, ارزیابی عملکرد, امتیاز کارایی",
    "پست سازمانی, سمت شغلی, مقام شغلی, موقعیت شغلی",
    "ضامن, تضمین, ضمانت‌نامه, ضامنان, تعهدنامه",
    "سفته, ضمانت‌نامه مالی, تعهد مالی, تضمین بازپرداخت",
    "اقساط, پرداخت‌های قسطی, اقساط ماهانه, بازپرداخت",
    "بودجه, منابع مالی, سرمایه, سقف بودجه, بودجه مصوب",
    "کمیته تسهیلات, گروه بررسی تسهیلات, هیئت تسهیلات, تیم تسهیلات",
    "وام, تسهیلات, قرض, اعتبار, اعتبارات مالی",
    "پرداخت وام, اعطای وام, اعطای تسهیلات, پرداخت تسهیلات",
    "مدت بازپرداخت, زمان بازپرداخت, دوره بازپرداخت, مدت اقساط",
    "سقف تسهیلات, حداکثر تسهیلات, میزان وام, حد وام",
    "امتیاز شغلی, نمره شغلی, امتیاز سمت, امتیاز کارمندی",
    "کارکنان تمام وقت, نیروی تمام وقت, کارکنان دائمی, پرسنل دائم",
    "کارمندان پاره وقت, نیروی پاره وقت, کارکنان غیر دائمی, پرسنل موقت",
    "کمیته بررسی, هیئت بررسی, تیم ارزیابی, گروه بررسی",
    "خدمات درمانی, هزینه‌های درمانی, پوشش درمانی, هزینه‌های پزشکی",
    "سقف وام, سقف تسهیلات, حد تسهیلات, سقف پرداختی",
    "کمک‌هزینه, تسهیلات مالی, حمایت مالی, کمک‌های مالی",
    "گواهی, مدرک, سند, تاییدیه, تصدیق",
    "برنامه بازپرداخت, جدول بازپرداخت, برنامه قسط‌بندی, نقشه بازپرداخت",
    "تسویه بدهی, پرداخت بدهی, تسویه حساب, پرداخت اقساط",
    "بازپرداخت وام, پرداخت قسط, پرداخت تسهیلات, تسویه وام",
    "شرایط پرداخت, قوانین پرداخت, شرایط تسهیلات, مقررات تسهیلات",
    "تسهیلات مسکن, وام مسکن, تسهیلات خرید مسکن, وام خرید خانه",
    "مدت بازپرداخت, دوره بازپرداخت, زمان تسویه, مدت قسط بندی"
]

In [15]:
all_synonyms = synonyms_list + synonyms
len(all_synonyms)

19928

In [16]:
all_valid = all(isinstance(syn, str) and ',' in syn for syn in synonyms_list)
if all_valid:
    print("All synonym rules are correctly formatted for Elasticsearch.")
else:
    print("Some rules may not be correctly formatted.")

All synonym rules are correctly formatted for Elasticsearch.


In [17]:
for i, syn in enumerate(synonyms_list):
    if not isinstance(syn, str) or ',' not in syn:
        print(f"Formatting issue detected in rule {i}: {syn}")

In [20]:
syn_index_config = {
    "settings": {
        "analysis": {
            "tokenizer": {
                "synonym_tokenizer": {
                    "type": "standard"
                }
            },
            "analyzer": {
                "synonym_analyzer": {
                    "type": "custom",
                    "tokenizer": "synonym_tokenizer",
                    "filter": ["synonym_filter"]
                }
            },
            "filter": {
                "synonym_filter": {
                    "type": "synonym",
                    "synonyms": all_synonyms
                }
            }
        }
    },
    "mappings": {
        "properties": {
            "content": {"type": "text", "analyzer": "synonym_analyzer"}
        }
    }
}

In [22]:
try:
    es.indices.create(index=index_name_syn, body=syn_index_config)
    print(f"--------------------- Index {index_name_syn} created successfully ---------------------")
except Exception as e:
    print(e)

--------------------- Index finance_department_syn created successfully ---------------------


In [23]:
for i, doc in enumerate(data):
    doc_body = {
        "content": doc,
    }
    unique_id = str(i)
    es.index(index=index_name_syn, id=unique_id, body=doc_body, refresh='wait_for')
    print(i)
print("--------------------- index successfully created ---------------------")

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
--------------------- index successfully created ---------------------
